# Union-Find木

参考 \
[https://note.nkmk.me/python-union-find/](https://note.nkmk.me/python-union-find/)
[https://amateur-engineer-blog.com/union-find/?utm_source=rss&utm_medium=rss&utm_campaign=union-find](https://amateur-engineer-blog.com/union-find/?utm_source=rss&utm_medium=rss&utm_campaign=union-find)

## 実装
note.nkmk.meから引用

In [3]:
from collections import defaultdict

class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            # 親要素を再帰的に探索
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        if self.parents[x] > self.parents[y]:
            x, y = y, x

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        group_members = defaultdict(list)
        for member in range(self.n):
            group_members[self.find(member)].append(member)
        return group_members

    def __str__(self):
        return '\n'.join(f'{r}: {m}' for r, m in self.all_group_members().items())

In [5]:
uf = UnionFind(6)
print(uf.parents)
print(uf)

[-1, -1, -1, -1, -1, -1]
0: [0]
1: [1]
2: [2]
3: [3]
4: [4]
5: [5]


In [6]:
uf.union(0, 2)
print(uf.parents)
print(uf)

[-2, -1, 0, -1, -1, -1]
0: [0, 2]
1: [1]
3: [3]
4: [4]
5: [5]


In [7]:
uf.union(1, 3)
print(uf.parents)
uf.union(4, 5)
print(uf.parents)
uf.union(1, 4)
print(uf.parents)

print(uf)

[-2, -2, 0, 1, -1, -1]
[-2, -2, 0, 1, -2, 4]
[-2, -4, 0, 1, 1, 4]
0: [0, 2]
1: [1, 3, 4, 5]


In [8]:
uf.union(0, 1)
print(uf.parents)
print(uf)

[1, -6, 0, 1, 1, 1]
1: [0, 1, 2, 3, 4, 5]


## 素朴な実装

In [42]:
class UnionFindBasic():
    def __init__(self, n):
        self.parents = list(range(n))

    def find(self, x):
        if self.parents[x] == x:
            return x
        else:
            print(x, end=" -> ")
            return self.find(self.parents[x])

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        self.parents[y] = x

In [43]:
ufb = UnionFindBasic(7)

ufb.union(5, 6)
ufb.union(4, 5)
ufb.union(3, 4)
ufb.union(2, 3)
ufb.union(1, 2)
print(ufb.parents)

[0, 1, 1, 2, 3, 4, 5]


In [44]:
print(ufb.find(5))
print(ufb.find(5))

5 -> 4 -> 3 -> 2 -> 1
5 -> 4 -> 3 -> 2 -> 1


### 経路圧縮

In [31]:
class UnionFindPathCompression():
    def __init__(self, n):
        self.parents = list(range(n))

    def find(self, x):
        if self.parents[x] == x:
            return x
        else:
            print(x, end=" -> ")
            self.parents[x] = self.find(self.parents[x]) # メモ化
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        self.parents[y] = x

In [46]:
uf_comp = UnionFindPathCompression(7)

uf_comp.union(5, 6)
uf_comp.union(4, 5)
uf_comp.union(3, 4)
uf_comp.union(2, 3)
uf_comp.union(1, 2)
print(uf_comp.parents)

[0, 1, 1, 2, 3, 4, 5]


In [47]:
print(uf_comp.find(5))
print(uf_comp.find(5))
# 2回目からの呼び出しが大幅に効率化

5 -> 4 -> 3 -> 2 -> 1
5 -> 1


### rankの導入

> ランク（木の高さ）の情報を保持しておき、併合する際に低い方を高い方につなげる（低い方の根の親を高い方の根にする）。

> 経路圧縮にランクを加えた実装は以下の通り。ランク情報を格納するリストであるrank属性を追加する。[0] * nで初期化する。

In [65]:
class UnionFindByRank():
    def __init__(self, n):
        self.parents = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parents[x] == x:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        if self.rank[x] < self.rank[y]:
            self.parents[x] = y
        else:
            self.parents[y] = x
            if self.rank[x] == self.rank[y]:
                self.rank[x] += 1